# TT recipes: Sensitivity Analysis Examples

This notebook shows how to use the __sensibility analysis__ methods from __TT tecipes__. These examples use functions from the built-in library of well-known test functions included in the submodule _models_. Some of these examples have been used for related scientific publications:

  - [1] _Sobol Tensor Trains for Global Sensitivity Analysis._
    Rafael Ballester-Ripoll, Enrique G. Paredes, Renato Pajarola
    [[arXiv](https://arxiv.org/abs/1712.00233)]

  - [2] _Tensor Approximation of Advanced Metrics for Sensitivity Analysis._
    Rafael Ballester-Ripoll, Enrique G. Paredes, Renato Pajarola
    [[arXiv]()]

In [ ]:
import pprint
import random

import numpy as np

# The most important "import" statement!
import ttrecipes as tr

Let's set some general settings for the examples in this tutorial:

  - __bins__ will be used as the default number of samples per axis
  - __max_order__ indicates the maximum order of the indices reported (all possible orders are always computed, this only affects the report tables)
  - __verbose__ sets the _verbose_ mode to __True__ or __False__
  - __seed__ is the random seed that will be used for Python and NumPy's random generators


In [ ]:
# Number of samples per axis
bins = 100

# Maximum order of the shown sensitivity indices 
max_order = 2

# Verbose mode (True/False)
verbose = True

# Set a seed value (e.g. 9999) if you want to get
# always the same results 
seed = None  # 9999
if seed is not None:
    random.seed(seed)
    np.random.seed(seed)

## Piston function

We will use as a first example a function modeling the circular motion of a piston within a cylinder with a chain of nonlinear functions:

\begin{equation}
f(\vec{x}) = 2 \pi \sqrt{\frac{M}{k + S^2 \frac{P_0 V_0}{T_0} \frac{T_a}{V^2}}}
\end{equation}

with:

\begin{equation}
\begin{gathered}
V = \frac{S}{2k} \left(\sqrt{A^2 + 4k \frac{P_0 V_0}{T_0} T_a} - A\right) \\
A = P_0 S + 19.62 M - \frac{k V_0}{S}
\end{gathered}
\end{equation}

More details about the function can be found at the [Virtual Library of Simulation Experiments](https://www.sfu.ca/~ssurjano/piston.html) from [Simon Fraser University](https://www.sfu.ca).

The first step is to get the function object and the axis definitions from the library of test models included in TT recipes:

In [ ]:
# Get example function and axes definitions from 'ttrecipes.models'
f_piston, axes_piston = tr.models.get_piston()
pprint.pprint(axes_piston)

Note that the second parameter (_number of samples_) of every axes definitions is set to _'None'_. This means that the number of samples is not explicitly fixed and the default value will be used when calling to the function computing the sensitivity indices (__tr.sensitivity_analysis.var_metrics()__). The exact meaning of each one of the parameters is explained in the function documentation:

In [ ]:
help(tr.sensitivity_analysis.var_metrics)

And now let's compute and show the actual indices. With __TT recipes__ this is as simple as calling the __tr.sensitivity_analysis.var_metrics()__ function with the function object and the axis definitions. In this example we set the parameter __print_results__ to __True__ to get the results tables printed.

All the other parameters are optional and control different settings, as the default number of samples per axes when it is not specifically set for every axis, the target relative error of the approximation, and more specific settings for the TT cross approximation algorithm. Check the documentation of the function in the previous cell for a detailed description.

In [ ]:
# Compute sensitivity indices and print result tables
print("+ Computing tensor approximations of variance-based sensitivity metrics...")
results = tr.sensitivity_analysis.var_metrics(
    f_piston, axes_piston, default_bins=bins, verbose=verbose, eps=1e-5,
    max_order=max_order, cross_kwargs=dict(),
    print_results=True)

By default only indices up to second order are printed. If we are interested in higher order interactions, we can get them by changing the value of the *max_order* parameter:

In [ ]:
# Compute sensitivity indices and print result tables
print("+ Computing tensor approximations of variance-based sensitivity metrics...")
results = tr.sensitivity_analysis.var_metrics(
    f_piston, axes_piston, default_bins=bins, verbose=verbose, eps=1e-5,
    max_order=3, cross_kwargs=dict(),
    print_results=True)

## Sobol __G__ function

Let's try now a more challenging example using the __Sobol _G_ __ function, which is a well-known synthetic model that can be tune for generating low or high-order interactions depending on the vector of coefficients __a_i__. A much more detailed description of this function can be also found at the [Virtual Library of Simulation Experiments](https://www.sfu.ca/~ssurjano/gfunc.html).

In this case, we will run the example with the same function parameters used in [2]:
  - **dimensions**: 20
  - **a_i**: 0 _[for i in range(n)]_

Again, the first step is to get the function object and the axis definitions from models library:

In [ ]:
# Set example parameters
dims = 20
acoeff = 0

# Get example function and axes definitions from 'ttrecipes.models'
f_sobolg, axes_sobolg = tr.models.get_sobol_g(dims, a=acoeff, name_tmpl='X_{:0>2}')
pprint.pprint(axes_sobolg)

Once the function and axes have been defined for our specific case, we just need to pass them as parameters to the __tr.sensitivity_analysis.var_metrics()__  function to get all the sensitivity metrics:

In [ ]:
# Compute sensitivity indices and print result tables
print("+ Computing tensor approximations of variance-based sensitivity metrics...")
results = tr.sensitivity_analysis.var_metrics(
    f_sobolg, axes_sobolg, default_bins=bins, verbose=verbose, eps=1e-10,
    max_order=max_order, cross_kwargs=dict(kickrank=2),
    print_results=True)

## Simulated decay chain

The third example simulates a radioactive decay chain that concatenates Poisson processes (it is a linear Jackson network) for 11 chemical species. Each species (except the last one) can decay into the next species in the chain.

This model has 10 parameters, namely the decay rates $\lambda_n$ of the 10 first species. The result of simulation $f_T(\lambda_1, \dots, \lambda_{10})$ is the amount of stable material (last node in the chain) measured after a certain time span $T$. The function $f_T$ simulates the decay chain by discretizing the span $T$ into timesteps of one day. The $\lambda_n$ (decay rates) represent the fraction of each material that decays every day. They are independent and uniformly distributed in the interval $[0.00063281, 0.00756736]$, which corresponds to half-lives from 3 years down to 3 months. The simulated time span for this example is set to 2 years ($T = 2$) as in [2]:

In [ ]:
# Set example parameters
span = 2
products = 10

f_decay, axes_decay = tr.models.get_decay_poisson(
    d=products, span=span, hl_range=(3.0 * (1 / 12.0), 3.0),
    time_step=1.0 / 365.0, name_tmpl='lambda_{:0>2}')
pprint.pprint(axes_decay)

print("+ Computing tensor approximations of variance-based sensitivity metrics...")
results = tr.sensitivity_analysis.var_metrics(
    f_decay, axes_decay, default_bins=bins, verbose=verbose, eps=1e-4,
    max_order=max_order, print_results=True)


## Fire-spread model

In the last example we use a function to model the rate of fire spread in the Mediterranean shrublands according to 10 variables that are fed into Rothermel's equations from [3]. We use the updated equations for the model from [4] since the authors kindly provided us the code of their function.

[3] _A mathematical model for predicting fire spread in wildland fuels (Tech.
Report)._ R. C. Rothermel. U.S. Department of Agriculture, Intermountain Forest and Range Experiment Station, 1972.

[4] _Shapley effects for global sensitivity analysis: Theory
and computation._ E. Song, B. L. Nelson, and J. Staum. SIAM/ASA Journal on Uncertainty Quantification, 4 (2016)

In [ ]:
f_fire, axes_fire = tr.models.get_fire_spread(wind_factor=6.9)
pprint.pprint(axes_fire)

print("+ Computing tensor approximations of variance-based sensitivity metrics...")
results = tr.sensitivity_analysis.var_metrics(
    f_fire, axes_fire, default_bins=bins, verbose=verbose, eps=1e-4,
    max_order=max_order, cross_kwargs=dict(kickrank=4), print_results=True)
